<a href="https://colab.research.google.com/github/walkerjian/DailyCode/blob/main/URLShortener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Problem:
Implement a URL shortener with the following methods:

shorten(url), which shortens the url into a six-character alphanumeric string, such as zLg6wl.
restore(short), which expands the shortened string into the original url. If no such shortened string exists, return null.
Hint: What if we enter the same URL twice?

##Solution:
To implement a URL shortener in Python, we need to take into account that the same URL should return the same shortened version each time it's entered, and we must be able to map back the shortened version to the original URL. A common approach to this is to use a hash function, but since hash functions can produce outputs larger than six characters and don't guarantee the same output length, we'll create a custom method.

##Implementation:
This code includes a basic collision handling mechanism which tries to find a unique hash for each URL. However, note that with a six-character limit, there is a possibility of collision which increases as more URLs are shortened. In real-world applications, you would likely need to include additional layers of checks and potentially increase the length of the shortened string to reduce collision risk.

In [1]:
import string
import hashlib
from random import choice

class URLShortener:
    def __init__(self):
        self.url_to_short = {}
        self.short_to_url = {}
        self.base = string.ascii_letters + string.digits  # base62 [a-zA-Z0-9]
        self.prefix = "http://short.url/"
        self.short_length = 6

    def _generate_short(self, url):
        # Use a hash function to generate a unique output
        sha_signature = hashlib.sha256(url.encode()).hexdigest()
        # Take the first 6 characters of the hash as our short url
        for i in range(0, len(sha_signature), self.short_length):
            short = sha_signature[i:i+self.short_length]
            if short not in self.short_to_url:
                return short
        # If all possible combinations are taken, raise an error (very unlikely with SHA256)
        raise ValueError("No unique short URLs available.")

    def shorten(self, url):
        if url in self.url_to_short:
            # If the URL has already been shortened, return the existing short URL
            return self.prefix + self.url_to_short[url]

        # Generate a unique short path
        short = self._generate_short(url)
        if short not in self.short_to_url:
            # Store the short URL and the original URL in dictionaries
            self.short_to_url[short] = url
            self.url_to_short[url] = short
            return self.prefix + short
        else:
            # In case of a hash collision, try another random short string (this is extremely unlikely with SHA256)
            while short in self.short_to_url:
                short = ''.join(choice(self.base) for _ in range(self.short_length))
            self.short_to_url[short] = url
            self.url_to_short[url] = short
            return self.prefix + short

    def restore(self, short):
        # Remove the prefix to get the short code
        short = short.replace(self.prefix, '')
        # Lookup the short code in the dictionary to get the original URL
        return self.short_to_url.get(short, None)


# Example of usage:
url_shortener = URLShortener()
original_url = "https://www.example.com"
short_url = url_shortener.shorten(original_url)
print(f"Short URL: {short_url}")

restored_url = url_shortener.restore(short_url)
print(f"Restored URL: {restored_url}")


Short URL: http://short.url/cdb4d8
Restored URL: https://www.example.com
